# Исходные данные

Данные, с которыми проводится работа - таблица формата CSV.

 В таблице присутствует следующая информация:
 - userID — идентификатор пользователя;
 - Track — название трека;
 - artist — имя исполнителя;
 - genre — название жанра;
 - City — город пользователя;
 - time — время начала прослушивания;
 - Day — день недели.

# Задачи исследования
В проекте будет проверено 3 гипотезы для Москвы и Санкт-Петербурга:

1. Активность пользователей зависит от дня недели (понедельник, среда, воскресенье). 
2. Есть существенная разница в предпочтении жанров по разным дням недели - понедельнику и пятнице.
3. Москва и Петербург предпочитают разные жанры музыки.

# Ход исследования

## Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

# чтение файла с данными и его сохранение в переменной df
df = pd.read_csv('yandex_music_project.csv')

In [2]:
# получение первых 5 строк таблицы df
display(df.head())

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [3]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


<div class="alert alert-info">
    
Находим нарушения стиля:
 - Строчные буквы сочетаются с прописными.
 - Встречаются пробелы.
 - Нет нижних подчеркиваний (UserID).
    
Находим наличие пропусков данных в столбцах.
    
</div>

## Предобработка данных

### Заголовки

In [4]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [5]:
# переименование столбцов
df = df.rename(columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

# проверка результатов - перечень названий столбцов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски

In [6]:
# подсчёт пропусков по столбцам
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [7]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')
    
# проверочный подсчёт пропусков
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

In [8]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [9]:
# удаление явных дубликатов и повторная индексация
df = df.drop_duplicates().reset_index(drop=True)

# проверка на отсутствие дубликатов
df.duplicated().sum() 

0

In [10]:
# Просмотр уникальных названий жанров
sorted_genre_df = df['genre']
sorted_genre_df = sorted_genre_df.sort_values()
sorted_genre_df.unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [12]:
# Устранение неявных дубликатов жанра хип-хоп
previous = ['hip', 'hop', 'hip-hop']
df['genre'] = df['genre'].replace(previous, 'hiphop')

# Повторная проверка на неявные дубликаты
sorted_df = df['genre']
sorted_df = sorted_df.sort_values()
sorted_df.unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

##  Проверка гипотез

### Активность пользователей в зависимости от дня недели. 

In [13]:
# Подсчёт прослушиваний в каждом городе
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

In [14]:
# Подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

<div class="alert alert-info">
    
Объявим функцию для подсчёта прослушиваний для конкретного города и дня.
    
С помощью последовательной фильтрации с логической индексацией она:
 - сначала получит из исходной таблицы строки с нужным днём, 
 - затем из результата отфильтрует строки с нужным городом,
 - затем посчитает количество слушателей. 
    
Это количество функция вернёт в качестве результата.
    
</div>    

In [16]:
def number_tracks(day,city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [20]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [21]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [22]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [23]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [24]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [42]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

In [43]:
# Таблица с результатами
data = {'city' : ['Moscow', 'Saint-Petersburg'], 'monday' : [15740, 5614], 'wednesday' : [11056, 7003], 'friday' : [15945, 5895]} 

pd.DataFrame(data=data)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


<div class="alert alert-success" role="alert">
    
Несмотря на то, что пользователей Москвы в принципе больше (как и населения), появляется явная тенденция:
    Пик прослушиваний в Москве приходится на понедельник и пятницу, а в Петербурге на среду.

### Преобладание разных жанров в зависимости от дня недели

In [27]:
# получение таблицы moscow_general из таблицы df, где значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']
display(moscow_general.head())

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday


In [28]:
# получение таблицы spb_general из таблицы df, где значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']
display(spb_general.head())

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


<div class="alert alert-info">

Объявим функцию, которая вернет информацию о самых популярных жанрах в указанный день в заданное время.

В функции:
- сохраним в переменную строки переданного датафрейма, для которых одновременно день равен day, время меньше time2 и больше time1
- сгруппируем полученный датафрейм по столбцу жанрам и вычислим количество записей для каждого жанра 
- отсортируем полученный датафрейм по убыванию встречаемости
- выведем топ-10 популярных жанров (в указанный день, в заданное время)

In [30]:
def genre_weekday(df, day, time1, time2):
    
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df = genre_df[genre_df['time'] > time1]
    
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending = False)
    
    return genre_df_sorted[:10] 

Сравненим результаты по городам. 

Учтем, что:
  - Утро - 7-11; 
  - Вечер - 17-23.

In [31]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [32]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [33]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [34]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

<div class="alert alert-success" role="alert">

Топ-3 жанров выглядит следующим образом:
    
**Понедельник**
 - Москва - поп, дэнс, электро; 
 - Питер - поп, дэнс, рок.

**Пятница**
 - Москва - поп, рок, дэнс; 
 - Питер - поп, электро, рок.
    
    Сделать вывод о том, что существует существенная разница в предпочтении разных жанров у москвичей и петербуржцев в определенные дни недели нельзя.

### Предпочтение разных жанров по городам.

In [38]:
# Группировка таблицы moscow_general по столбцу 'genre', 
# Затем подсчёт числа значений 'genre' и сортировка получившегося Series в порядке убывания 

moscow_genres = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_genres.sort_values(ascending = False)

In [39]:
# просмотр первых 10 строк moscow_genres
print(moscow_genres[:10])

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [40]:
# Группировка таблицы spb_general по столбцу 'genre', 
# Затем подсчёт числа значений 'genre' и сортировка получившегося Series в порядке убывания 

spb_genres = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_genres.sort_values(ascending = False)

In [41]:
# просмотр первых 10 строк spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


<div class="alert alert-success" role="alert">

Топ-5 жанров по городам выглядит следующий образом:
    
  **Москва**
  1. Поп
  2. Дэнс
  3. Рок
  4. Электро
  5. Хип-хоп
    
  **Петербург**  
  1. Поп
  2. Дэнс
  3. Рок
  4. Электро
  5. Хип-хоп
    
Топ-5 по городам абсолютно индентичны, разница в предпочтениях несущественно начинает проявляться только после 5 самыых популярных жанров. Сделать вывод, что 3-я гипотеза правдива нельзя.

# Итоги

<div class="alert alert-danger" role="alert">

Было проверено 3 гипотезы и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.
Первая гипотеза полностью подтвердилась.
2. Преобладание разных жанров в зависимости от дня недели.
Музыкальные предпочтения не сильно меняются в течение недели, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы в данных не было пропусков.
3. Предпочтение разных жанров по городам. 
Третья гипотеза не подтвердилась. Топ-5 жанров по двум городам индентичен. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.